# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
# Import Lib
import os
import pandas as pd
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Your code here
df = pd.read_csv("./homepage_actions.csv")
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
timestamp    8188 non-null object
id           8188 non-null int64
group        8188 non-null object
action       8188 non-null object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [4]:
df.describe()

,id
count,8188.000000
mean,564699.749878
std,219085.845672
min,182988.000000
25%,373637.500000
50%,566840.500000
75%,758078.000000
max,937217.000000


In [5]:
df["action"].unique()

array(['view', 'click'], dtype=object)

In [6]:
df["group"].unique()

array(['experiment', 'control'], dtype=object)

In [7]:
df["action"].value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [8]:
exp_grp = df[df["group"] == "experiment"]
print(len(exp_grp))
exp_grp.head()

3924


,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
5,2016-09-24 20:32:25.712659,681598,experiment,view


In [9]:
ctrl_grp = df[df["group"] == "control"]
print(len(ctrl_grp))
ctrl_grp.head()

4264


,timestamp,id,group,action
3,2016-09-24 19:59:02.646620,671993,control,view
9,2016-09-24 21:00:12.278374,560027,control,view
23,2016-09-25 00:25:14.141290,281985,control,view
24,2016-09-25 01:14:48.603202,407864,control,view
25,2016-09-25 02:16:11.046654,342984,control,view


In [10]:
# Overlap of experiment and control groups?
eids = set(df[df.group=='experiment']['id'].unique())
cids = set(df[df.group=='control']['id'].unique())
print('Overlap of experiment and control groups: {}'.format(len(eids&cids)))

Overlap of experiment and control groups: 0


In [11]:
# How many viewers also clicked?
cids = set(df[df["action"]=='click']['id'].unique())
vids = set(df[df["action"]=='view']['id'].unique())
print (len(cids), len(vids))

# Are there any anomalies with the data; did anyone click who didn't view?
print("Number of viewers: {} \tNumber of clickers: {}".format(len(vids), len(cids)))
print("Number of Viewers who didn't click: {}".format(len(vids-cids)))
print("Number of Clickers who didn't view: {}".format(len(cids-vids)))

1860 6328
Number of viewers: 6328 	Number of clickers: 1860
Number of Viewers who didn't click: 4468
Number of Clickers who didn't view: 0


In [12]:
ctrl_grp_view = df[(df["group"] == "control") & (df["action"] == "view")]
print(len(ctrl_grp_view))
ctrl_grp_view.head()

3332


,timestamp,id,group,action
3,2016-09-24 19:59:02.646620,671993,control,view
9,2016-09-24 21:00:12.278374,560027,control,view
23,2016-09-25 00:25:14.141290,281985,control,view
24,2016-09-25 01:14:48.603202,407864,control,view
25,2016-09-25 02:16:11.046654,342984,control,view


In [13]:
ctrl_grp_click = df[(df["group"] == "control") & (df["action"] == "click")]
print(len(ctrl_grp_click))
ctrl_grp_click.head()

932


,timestamp,id,group,action
28,2016-09-25 02:53:25.459874,398892,control,click
37,2016-09-25 05:19:15.810727,544571,control,click
56,2016-09-25 08:25:32.821891,194950,control,click
58,2016-09-25 09:45:12.114972,894454,control,click
63,2016-09-25 10:38:53.299877,639852,control,click


In [14]:
exp_grp_click = df[(df["group"] == "experiment") & (df["action"] == "click")]
print(len(exp_grp_click))
exp_grp_click.head()

928


,timestamp,id,group,action
8,2016-09-24 20:58:01.948663,349125,experiment,click
12,2016-09-24 21:06:27.553057,601714,experiment,click
15,2016-09-24 21:30:02.739756,487634,experiment,click
17,2016-09-24 23:01:12.108316,468601,experiment,click
21,2016-09-25 00:01:47.933853,555973,experiment,click


In [15]:
exp_grp_view = df[(df["group"] == "experiment") & (df["action"] == "view")]
print(len(exp_grp_view))
exp_grp_view.head()

2996


,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
5,2016-09-24 20:32:25.712659,681598,experiment,view


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

### Using get_dummies()

In [16]:
df.columns

Index(['timestamp', 'id', 'group', 'action'], dtype='object')

In [17]:
dummy_action_cols = pd.get_dummies(df["action"])
dummy_action_cols
df = pd.concat([df, dummy_action_cols], axis=1)
df.head(10)

#dummies = pd.get_dummies(df['Category'])

,timestamp,id,group,action,click,view
0,2016-09-24 17:42:27.839496,804196,experiment,view,0,1
1,2016-09-24 19:19:03.542569,434745,experiment,view,0,1
2,2016-09-24 19:36:00.944135,507599,experiment,view,0,1
3,2016-09-24 19:59:02.646620,671993,control,view,0,1
4,2016-09-24 20:26:14.466886,536734,experiment,view,0,1
5,2016-09-24 20:32:25.712659,681598,experiment,view,0,1
6,2016-09-24 20:39:03.248853,522116,experiment,view,0,1
7,2016-09-24 20:57:20.336757,349125,experiment,view,0,1
8,2016-09-24 20:58:01.948663,349125,experiment,click,1,0
9,2016-09-24 21:00:12.278374,560027,control,view,0,1


In [18]:
ctrl_grp_dmmy = df[(df["group"] == "control") ]
print(len(ctrl_grp_dmmy))
ctrl_grp_dmmy.head()

4264


,timestamp,id,group,action,click,view
3,2016-09-24 19:59:02.646620,671993,control,view,0,1
9,2016-09-24 21:00:12.278374,560027,control,view,0,1
23,2016-09-25 00:25:14.141290,281985,control,view,0,1
24,2016-09-25 01:14:48.603202,407864,control,view,0,1
25,2016-09-25 02:16:11.046654,342984,control,view,0,1


In [19]:
exp_grp_dmmy = df[(df["group"] == "experiment") ]
print(len(exp_grp_dmmy))
exp_grp_dmmy.head()

3924


,timestamp,id,group,action,click,view
0,2016-09-24 17:42:27.839496,804196,experiment,view,0,1
1,2016-09-24 19:19:03.542569,434745,experiment,view,0,1
2,2016-09-24 19:36:00.944135,507599,experiment,view,0,1
4,2016-09-24 20:26:14.466886,536734,experiment,view,0,1
5,2016-09-24 20:32:25.712659,681598,experiment,view,0,1


In [20]:
df_exp_grp_click = df[(df["group"] == "experiment") & (df["action"] == "click")]
print(len(df_exp_grp_click))
df_exp_grp_click.head()

928


,timestamp,id,group,action,click,view
8,2016-09-24 20:58:01.948663,349125,experiment,click,1,0
12,2016-09-24 21:06:27.553057,601714,experiment,click,1,0
15,2016-09-24 21:30:02.739756,487634,experiment,click,1,0
17,2016-09-24 23:01:12.108316,468601,experiment,click,1,0
21,2016-09-25 00:01:47.933853,555973,experiment,click,1,0


In [21]:
df_ctrl_grp_click = df[(df["group"] == "control") & (df["action"] == "click")]
print(len(df_ctrl_grp_click))
df_ctrl_grp_click.head()

932


,timestamp,id,group,action,click,view
28,2016-09-25 02:53:25.459874,398892,control,click,1,0
37,2016-09-25 05:19:15.810727,544571,control,click,1,0
56,2016-09-25 08:25:32.821891,194950,control,click,1,0
58,2016-09-25 09:45:12.114972,894454,control,click,1,0
63,2016-09-25 10:38:53.299877,639852,control,click,1,0


In [22]:
df_ctrl_grp_click["click"].head(10)

28     1
37     1
56     1
58     1
63     1
67     1
70     1
81     1
86     1
112    1
Name: click, dtype: uint8

In [23]:
print("Sample sizes:\tControl: {}\tExperiment: {}".format(len(ctrl_grp), len(exp_grp)))
print("Total Clicks:\tControl: {}\tExperiment: {}".format(np.sum(df_ctrl_grp_click["click"]), np.sum(df_exp_grp_click["click"])))

# CTR is the number of clicks that their ad receives divided by the number of times their ad is viewed: clicks ÷ views = CTR.
print("Average click rate:\tControl: {}\tExperiment: {}".format(np.sum(df_ctrl_grp_click["click"]/3332), np.sum(df_exp_grp_click["click"]/ 2996)))
#control.head()

Sample sizes:	Control: 4264	Experiment: 3924
Total Clicks:	Control: 932	Experiment: 928
Average click rate:	Control: 0.2797118847539015	Experiment: 0.30974632843791716


In [24]:
import flatiron_stats as fs
fs.p_value_welch_ttest(ctrl_grp_dmmy["click"], exp_grp_dmmy["click"])   # 0.004466402814337078

0.026743886922199422

### Not using get_dummies()

In [25]:
exp_click_count = exp_grp_click["action"].value_counts()
exp_click_count[0]

928

In [26]:
ctrl_click_count = ctrl_grp_click["action"].value_counts()
ctrl_click_count[0]

932

In [27]:
print("Sample sizes:\tControl: {}\tExperiment: {}".format(len(ctrl_grp), len(exp_grp)))
print("Total Clicks:\tControl: {}\tExperiment: {}".format(ctrl_click_count[0], exp_click_count[0]))
print("Average click rate:\tControl: {}\tExperiment: {}".format(ctrl_click_count[0]/3332, exp_click_count[0]/2996))
# control.head()

Sample sizes:	Control: 4264	Experiment: 3924
Total Clicks:	Control: 932	Experiment: 928
Average click rate:	Control: 0.2797118847539016	Experiment: 0.3097463284379172


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

In [44]:
# Step 1:
# Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group.

control_rate = np.sum(df_ctrl_grp_click["click"]/3332)
expected_experiment_clicks_under_null = 838.0168067226891 # control_rate * len(exp_grp)
print(expected_experiment_clicks_under_null)  # 838.0168067226891

838.0168067226891


In [45]:
# Step 2:
# Calculate the number of standard deviations that the actual number of clicks was from this estimate.

n = len(exp_grp)
p = control_rate
var = n * p * (1-p)
std = 24.568547907005815 # np.sqrt(var)
print(std)    # 24.568547907005815

24.568547907005815


In [46]:
actual_experiment_clicks = np.sum(df_exp_grp_click["click"])
z_score = (actual_experiment_clicks - expected_experiment_clicks_under_null)/std
print(z_score)

3.6625360854823588


In [47]:
# Step 3:
# Finally, calculate a p-value using the normal distribution based on this z-score.

import scipy.stats as stats
p_val = stats.norm.sf(z_score) #or 1 - stats.norm.cdf(z_score)
print(p_val)

0.00012486528006951198


### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [28]:
#Your code here

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [29]:
#Your code here

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [30]:
#Your code here

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

In [48]:
# Comment: Yes, while the p-value is slightly lower, 
# both would lead to confident rejection of the null hypothesis. 
# The experimental page appears to be a more effective design.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.